In [1]:
### Hayden Gallo
### Bucci Lab
### Testing Mouse GEM for MDSINE-FBA
### 7/8/25

import numpy as np
#from dfba import DfbaModel, ExchangeFlux, KineticVariable
import cobra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from numba import njit
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
from scipy.optimize import curve_fit
import sys
import os
import openpyxl
import gurobipy

import pymc as pm
import pytensor
import pytensor.tensor as pt
from pymc.ode import DifferentialEquation
from pytensor.compile.ops import as_op
import arviz as az

import time
import joblib
import multiprocessing
from scipy.stats import truncnorm
import copy
from pathlib import Path
import subprocess

### script for running glv_dfba inference
from helper_functions import *
import argparse
import re

from matplotlib.patches import Patch
from matplotlib.backends.backend_pdf import PdfPages
import json

from Bio import Entrez

In [2]:
### load in mouse gem
mouse_model_bigg = cobra.io.read_sbml_model('/Users/haydengallo/UMass_Dropbox/Dropbox (UMass Medical School)/Bucci_Lab/glv_FBA/gLV_FBA_test_Kennedy_et_al_2025/processed_data_filtered_RC_all_cohorts_corrected_abs_abun/iMM1415.xml')
#mouse_model = cobra.io.read_sbml_model('/Users/haydengallo/UMass_Dropbox/Dropbox (UMass Medical School)/Bucci_Lab/glv_FBA/gLV_FBA_test_Kennedy_et_al_2025/processed_data_filtered_RC_all_cohorts_corrected_abs_abun/Mouse-GEM.xml')

mouse_model_bigg_base = mouse_model_bigg.copy()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-05-30


No objective coefficients in model. Unclear what should be optimized
https://identifiers.org/taxonomy/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


Read LP format model from file /var/folders/5n/y6k0c8y951n38r8ccc6trh1m0000gn/T/tmpilunjbg3.lp
Reading time = 0.01 seconds
: 2775 rows, 7452 columns, 28106 nonzeros


In [51]:
### Load in diet data

### Load in metabolomics data 

metabolomics_data = pd.read_csv('/Users/haydengallo/UMass_Dropbox/Dropbox (UMass Medical School)/Bucci_Lab/glv_FBA/gLV_FBA_test_Kennedy_et_al_2025/raw_data/complete_mapping_metabolomics.csv', index_col=0)
## fill NAs w/ zero 
metabolomics_data = metabolomics_data.fillna(0)

rc_diet_data = metabolomics_data[metabolomics_data['SampleName'] == 'RC_001']

rc_diet_MS_convert = rc_diet_data.copy()


with open('/Users/haydengallo/UMass_Dropbox/Dropbox (UMass Medical School)/Bucci_Lab/glv_FBA/gLV_FBA_test_Kennedy_et_al_2025/raw_data/BiGG_to_MSID.json') as f:
    bigg_to_modelseed = json.load(f)
    #print(bigg_to_modelseed)

cmpd_names = []
for i in rc_diet_data.columns.tolist()[1:]:
    cmpd_names.append(bigg_to_modelseed[i])


rc_diet_MS_convert = rc_diet_MS_convert.drop(columns=['SampleName'])
rc_diet_MS_convert.columns = cmpd_names
rc_diet_MS_convert = rc_diet_MS_convert.T
rc_diet_MS_convert

rc_diet_MS_convert['maxflux'] = 100
rc_diet_MS_convert = rc_diet_MS_convert.reset_index()
rc_diet_MS_convert.columns =['compounds','maxflux','minflux']
rc_diet_MS_convert['minflux'] = rc_diet_MS_convert['minflux']*-1.0
#rc_diet_MS_convert['minflux'] = -25
rc_diet_MS_convert

rc_diet_MS_convert = rc_diet_MS_convert[rc_diet_MS_convert['maxflux'] != 0.0]

rc_diet_MS_convert = rc_diet_MS_convert[['compounds', 'maxflux']]
rc_diet_MS_convert.columns = ['reaction', 'fluxValue']

### manually add 18 minimal metabolites to the RC_diet

mets_to_add = ['cpd00001','cpd00009','cpd00013','cpd00030','cpd00034','cpd00048','cpd00058','cpd00063','cpd00067','cpd00099','cpd00149','cpd00205','cpd00244','cpd00254','cpd00971','cpd10515','cpd10516','cpd11574','cpd00028']
flux = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]


mets_to_add_df = pd.DataFrame(mets_to_add, flux)
mets_to_add_df.reset_index(inplace=True)
mets_to_add_df.columns = ['fluxValue','reaction']
mets_to_add_df

rc_diet_MS_convert = pd.concat([rc_diet_MS_convert, mets_to_add_df])
rc_diet_MS_convert

,reaction,fluxValue
2,cpd01293,0.131
3,cpd00035,0.836
4,cpd01055,0.029
5,cpd00224,0.055
7,cpd00041,0.398
...,...,...
14,cpd00971,1.000
15,cpd10515,1.000
16,cpd10516,1.000
17,cpd11574,1.000


In [52]:
### load in the metabolite data for the mouse model

mouse_model_mets = pd.read_csv('/Users/haydengallo/UMass_Dropbox/Dropbox (UMass Medical School)/Bucci_Lab/glv_FBA/gLV_FBA_test_Kennedy_et_al_2025/processed_data_filtered_RC_all_cohorts_corrected_abs_abun/Mouse_GEM_metabolites.tsv', sep='\t')
mouse_model_mets.head()

,mets,metBiGGID,metChEBIID,metEHMNID,metHMDBID,metHMR2ID,metHepatoNET1ID,metKEGGID,metLipidMapsID,metMetaNetXID,metPubChemID,metRecon3DID,metRetired,metsNoComp
0,MAM00001c,carveol,CHEBI:15389,NaN,NaN,m00001c,NaN,C00964,NaN,MNXM45735,NaN,carveol,m00001c,MAM00001
1,MAM00001e,carveol,CHEBI:15389,NaN,NaN,m00001s,NaN,C00964,NaN,MNXM45735,NaN,carveol,m00001s,MAM00001
2,MAM00002c,appnn,CHEBI:36740,NaN,HMDB0006525,m00002c,NaN,C09880,NaN,MNXM163755,6654.0,appnn,m00002c,MAM00002
3,MAM00002e,appnn,CHEBI:36740,NaN,HMDB0006525,m00002s,NaN,C09880,NaN,MNXM163755,6654.0,appnn,m00002s,MAM00002
4,MAM00003c,NaN,CHEBI:78990,NaN,NaN,m00003c,NaN,NaN,LMFA01030283,MNXM150165;MNXM27815,NaN,M00003,m00003c,MAM00003


In [53]:
### reverse dict of bigg to modelseed

modelseed_to_bigg = {value: key for key, value in bigg_to_modelseed.items()}
modelseed_to_bigg['cpd00001'] = 'h2o'
### also convert cobalt correctly
modelseed_to_bigg['cpd00149'] = 'co2'
modelseed_to_bigg

{'cpd00001': 'h2o',
 'cpd00002': 'atp',
 'cpd00003': 'nad',
 'cpd00004': 'nadh',
 'cpd00005': 'nadph',
 'cpd00006': 'nadp',
 'cpd00007': 'o2',
 'cpd00008': 'adp',
 'cpd00009': 'pi',
 'cpd00010': 'coa',
 'cpd00011': 'co2',
 'cpd00012': 'ppi',
 'cpd00013': 'nh4',
 'cpd00014': 'udp',
 'cpd00015': 'fad',
 'cpd00016': 'pydx5p',
 'cpd00017': 'amet',
 'cpd00018': 'amp',
 'cpd00019': 'ahcys',
 'cpd00020': 'pyr',
 'cpd00022': 'accoa',
 'cpd00023': 'glu__L',
 'cpd00024': 'akg',
 'cpd00025': 'h2o2',
 'cpd00026': 'udpg',
 'cpd00027': 'glc__D',
 'cpd00028': 'pheme',
 'cpd00029': 'ac',
 'cpd00030': 'mn2',
 'cpd00031': 'gdp',
 'cpd00032': 'oaa',
 'cpd00033': 'gly',
 'cpd00034': 'zn2',
 'cpd00035': 'ala__L',
 'cpd00036': 'succ',
 'cpd00037': 'uacgam',
 'cpd00038': 'gtp',
 'cpd00039': 'lys__L',
 'cpd00040': 'glx',
 'cpd00041': 'asp__L',
 'cpd00042': 'gthrd',
 'cpd00043': 'udpgal',
 'cpd00044': 'paps',
 'cpd00045': 'pap',
 'cpd00046': 'cmp',
 'cpd00047': 'for',
 'cpd00048': 'so4',
 'cpd00050': 'fmn',
 '

In [54]:
RC_bigg_mets = []

for i in range(0, len(rc_diet_MS_convert)):
    RC_bigg_mets.append(modelseed_to_bigg[rc_diet_MS_convert['reaction'].iloc[i]])
RC_bigg_mets   

['5oxpro',
 'ala__L',
 'all__D',
 'arab__L',
 'asp__L',
 'cit',
 'cys__L',
 'dca',
 'dchac',
 'ddca',
 'fru',
 'fum',
 'gal',
 'glc__D',
 'glu__L',
 'gly',
 'hxa',
 'ile__L',
 'leu__L',
 'lnlcACP',
 'mal__L',
 'melib',
 'met__L',
 'ttdca',
 'nac',
 'octa',
 'hdca',
 'phe__L',
 'C10164',
 'pro__L',
 'pyr',
 'raffin',
 'rmn',
 'ser__D',
 'succ',
 'sucr',
 'thr__L',
 'tyr__L',
 'pta',
 'val__L',
 'cis_Oleic acid',
 'meso_Erythritol',
 'inost',
 '4hpro_LT',
 'h2o',
 'pi',
 'nh4',
 'mn2',
 'zn2',
 'so4',
 'cu2',
 'ca2',
 'h',
 'cl',
 'co2',
 'k',
 'ni2',
 'mg2',
 'na1',
 'fe2',
 'fe3',
 'mobd',
 'pheme']

In [55]:
RC_bigg_data = rc_diet_MS_convert.copy()
RC_mouse_mets = rc_diet_MS_convert.copy()
RC_bigg_data['reaction'] = RC_bigg_mets
RC_bigg_data

,reaction,fluxValue
2,5oxpro,0.131
3,ala__L,0.836
4,all__D,0.029
5,arab__L,0.055
7,asp__L,0.398
...,...,...
14,na1,1.000
15,fe2,1.000
16,fe3,1.000
17,mobd,1.000


In [142]:
'''
mouse_mets_list = []
for i in RC_bigg_mets:
    temp_df = mouse_model_mets[mouse_model_mets['metBiGGID'] == i]['mets']
    if temp_df.empty:
        mouse_mets_list.append('none')
    else: 
        print(temp_df)
        temp_str = mouse_model_mets[mouse_model_mets['metBiGGID'] == i]['mets'].iloc[0]
        mouse_mets_list.append(temp_str[:-1])
mouse_mets_list
'''

"\nmouse_mets_list = []\nfor i in RC_bigg_mets:\n    temp_df = mouse_model_mets[mouse_model_mets['metBiGGID'] == i]['mets']\n    if temp_df.empty:\n        mouse_mets_list.append('none')\n    else: \n        print(temp_df)\n        temp_str = mouse_model_mets[mouse_model_mets['metBiGGID'] == i]['mets'].iloc[0]\n        mouse_mets_list.append(temp_str[:-1])\nmouse_mets_list\n"

In [143]:
#RC_mouse_mets['reaction'] = mouse_mets_list
#RC_mouse_mets

In [58]:
#RC_mouse_mets_filt = RC_mouse_mets[RC_mouse_mets['reaction'] != 'none']
#RC_mouse_mets_filt

#RC_mouse_diet_dict = dict(zip(RC_mouse_mets_filt['reaction'], RC_mouse_mets_filt['fluxValue']))

In [59]:
#for key in RC_mouse_diet_dict.keys():
#    if key in list(mouse_model.medium.keys()):
#        print('yes')

In [60]:
mouse_model_bigg

Name,iMM1415
Memory address,314e4e390
Number of metabolites,2775
Number of reactions,3726
Number of genes,1375
Number of groups,0
Objective expression,1.0*BIOMASS_mm_1_no_glygln - 1.0*BIOMASS_mm_1_no_glygln_reverse_754f5
Compartments,"cytosol, extracellular space, golgi apparatus, lysosome, mitochondria, nucleus, endoplasmic reticulum, peroxisome/glyoxysome"


In [61]:
rxns = list(mouse_model_bigg.reactions)

In [62]:
biomass_pattern = re.compile(r'(bio)', re.IGNORECASE)
biomass_reactions = [rxn for rxn in mouse_model_bigg.reactions if biomass_pattern.search(rxn.id)]

In [63]:
biomass_reactions

[<Reaction EX_biocyt_e at 0x32ed59820>,
 <Reaction BIOCYTtn at 0x32f619d90>,
 <Reaction BIOMASS_mm_1_no_glygln at 0x330e9f0e0>]

In [64]:
mouse_model_bigg.objective

In [17]:
mouse_model_bigg.objective = {mouse_model_bigg.reactions.BIOMASS_mm_1_no_glygln: 1}
mouse_model_bigg.optimize()

,fluxes,reduced_costs
EX_5fthf_e,0.000000,0.000000e+00
EX_5homeprazole_e,0.000000,0.000000e+00
EX_34dhoxpeg_e,0.000000,0.000000e+00
EX_5htrp_e,0.000000,0.000000e+00
EX_34dhphe_e,0.000000,0.000000e+00
...,...,...
YVITEt,0.000000,0.000000e+00
BIOMASS_mm_1_no_glygln,1.363428,-9.831226e-16
R03184_PLUS_R00647,0.000000,-0.000000e+00
R01465m,0.473717,0.000000e+00


In [144]:
mouse_model_bigg.medium

{'EX_ala__L_e': 0.836,
 'EX_arab__L_e': 0.055,
 'EX_asp__L_e': 0.398,
 'EX_cys__L_e': 1.989,
 'EX_ca2_e': 1.0,
 'EX_cit_e': 1.046,
 'EX_cl_e': 1.0,
 'EX_co2_e': 1.0,
 'EX_octa_e': 0.13,
 'EX_fe2_e': 1.0,
 'EX_fe3_e': 1.0,
 'EX_phe__L_e': 0.115,
 'EX_pheme_e': 1.0,
 'EX_fru_e': 2.506,
 'EX_pi_e': 1.0,
 'EX_h_e': 1.0,
 'EX_pro__L_e': 2.307,
 'EX_h2o_e': 1.0,
 'EX_gal_e': 0.048,
 'EX_pyr_e': 0.032,
 'EX_glc__D_e': 0.612,
 'EX_glu__L_e': 0.017,
 'EX_gly_e': 0.072,
 'EX_hdca_e': 0.599,
 'EX_ser__D_e': 0.211,
 'EX_so4_e': 1.0,
 'EX_succ_e': 0.056,
 'EX_sucr_e': 32.513,
 'EX_ttdca_e': 0.696,
 'EX_tyr__L_e': 0.093,
 'EX_val__L_e': 0.121,
 'EX_thr__L_e': 0.033,
 'EX_ile__L_e': 0.387,
 'EX_inost_e': 0.502,
 'EX_k_e': 1.0,
 'EX_leu__L_e': 0.644,
 'EX_met__L_e': 4.781,
 'EX_na1_e': 1.0,
 'EX_nac_e': 0.172,
 'EX_nh4_e': 1.0}

In [66]:
RC_bigg_mets_convert = []
for i in RC_bigg_mets:
    temp = 'EX_' + i + '_e'
    RC_bigg_mets_convert.append(str(temp))
RC_bigg_mets_convert

['EX_5oxpro_e',
 'EX_ala__L_e',
 'EX_all__D_e',
 'EX_arab__L_e',
 'EX_asp__L_e',
 'EX_cit_e',
 'EX_cys__L_e',
 'EX_dca_e',
 'EX_dchac_e',
 'EX_ddca_e',
 'EX_fru_e',
 'EX_fum_e',
 'EX_gal_e',
 'EX_glc__D_e',
 'EX_glu__L_e',
 'EX_gly_e',
 'EX_hxa_e',
 'EX_ile__L_e',
 'EX_leu__L_e',
 'EX_lnlcACP_e',
 'EX_mal__L_e',
 'EX_melib_e',
 'EX_met__L_e',
 'EX_ttdca_e',
 'EX_nac_e',
 'EX_octa_e',
 'EX_hdca_e',
 'EX_phe__L_e',
 'EX_C10164_e',
 'EX_pro__L_e',
 'EX_pyr_e',
 'EX_raffin_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_succ_e',
 'EX_sucr_e',
 'EX_thr__L_e',
 'EX_tyr__L_e',
 'EX_pta_e',
 'EX_val__L_e',
 'EX_cis_Oleic acid_e',
 'EX_meso_Erythritol_e',
 'EX_inost_e',
 'EX_4hpro_LT_e',
 'EX_h2o_e',
 'EX_pi_e',
 'EX_nh4_e',
 'EX_mn2_e',
 'EX_zn2_e',
 'EX_so4_e',
 'EX_cu2_e',
 'EX_ca2_e',
 'EX_h_e',
 'EX_cl_e',
 'EX_co2_e',
 'EX_k_e',
 'EX_ni2_e',
 'EX_mg2_e',
 'EX_na1_e',
 'EX_fe2_e',
 'EX_fe3_e',
 'EX_mobd_e',
 'EX_pheme_e']

In [163]:
RC_bigg_data_convert_df = RC_bigg_data.copy()
RC_bigg_data_convert_df['reaction'] = RC_bigg_mets_convert

RC_dict_bigg_mouse = dict(zip(RC_bigg_data_convert_df['reaction'], RC_bigg_data_convert_df['fluxValue']))
RC_dict_bigg_mouse

{'EX_5oxpro_e': 0.131,
 'EX_ala__L_e': 0.836,
 'EX_all__D_e': 0.029,
 'EX_arab__L_e': 0.055,
 'EX_asp__L_e': 0.398,
 'EX_cit_e': 1.046,
 'EX_cys__L_e': 1.989,
 'EX_dca_e': 0.046,
 'EX_dchac_e': 0.028,
 'EX_ddca_e': 0.038,
 'EX_fru_e': 2.506,
 'EX_fum_e': 0.072,
 'EX_gal_e': 0.048,
 'EX_glc__D_e': 0.612,
 'EX_glu__L_e': 0.017,
 'EX_gly_e': 0.072,
 'EX_hxa_e': 0.059,
 'EX_ile__L_e': 0.387,
 'EX_leu__L_e': 0.644,
 'EX_lnlcACP_e': 5.732,
 'EX_mal__L_e': 0.382,
 'EX_melib_e': 0.021,
 'EX_met__L_e': 4.781,
 'EX_ttdca_e': 0.696,
 'EX_nac_e': 0.172,
 'EX_octa_e': 0.13,
 'EX_hdca_e': 0.599,
 'EX_phe__L_e': 0.115,
 'EX_C10164_e': 0.064,
 'EX_pro__L_e': 2.307,
 'EX_pyr_e': 0.032,
 'EX_raffin_e': 1.192,
 'EX_rmn_e': 0.027,
 'EX_ser__D_e': 0.211,
 'EX_succ_e': 0.056,
 'EX_sucr_e': 32.513,
 'EX_thr__L_e': 0.033,
 'EX_tyr__L_e': 0.093,
 'EX_pta_e': 0.027,
 'EX_val__L_e': 0.121,
 'EX_cis_Oleic acid_e': 3.842,
 'EX_meso_Erythritol_e': 0.064,
 'EX_inost_e': 0.502,
 'EX_4hpro_LT_e': 0.004,
 'EX_h2o_e': 1

In [162]:
temp_media = make_media(mouse_model_bigg, RC_bigg_data_convert_df)

In [161]:
mouse_model_bigg.medium = temp_media
mouse_model_bigg.optimize()

,fluxes,reduced_costs
EX_5fthf_e,0.0,1.776357e-15
EX_5homeprazole_e,0.0,0.000000e+00
EX_34dhoxpeg_e,0.0,0.000000e+00
EX_5htrp_e,0.0,0.000000e+00
EX_34dhphe_e,0.0,0.000000e+00
...,...,...
YVITEt,0.0,0.000000e+00
BIOMASS_mm_1_no_glygln,0.0,3.282069e-14
R03184_PLUS_R00647,0.0,8.881784e-16
R01465m,0.0,-1.060806e-15


In [160]:
RC_bigg_data_convert_df['reaction'].to_list()

['EX_5oxpro_e',
 'EX_ala__L_e',
 'EX_all__D_e',
 'EX_arab__L_e',
 'EX_asp__L_e',
 'EX_cit_e',
 'EX_cys__L_e',
 'EX_dca_e',
 'EX_dchac_e',
 'EX_ddca_e',
 'EX_fru_e',
 'EX_fum_e',
 'EX_gal_e',
 'EX_glc__D_e',
 'EX_glu__L_e',
 'EX_gly_e',
 'EX_hxa_e',
 'EX_ile__L_e',
 'EX_leu__L_e',
 'EX_lnlcACP_e',
 'EX_mal__L_e',
 'EX_melib_e',
 'EX_met__L_e',
 'EX_ttdca_e',
 'EX_nac_e',
 'EX_octa_e',
 'EX_hdca_e',
 'EX_phe__L_e',
 'EX_C10164_e',
 'EX_pro__L_e',
 'EX_pyr_e',
 'EX_raffin_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_succ_e',
 'EX_sucr_e',
 'EX_thr__L_e',
 'EX_tyr__L_e',
 'EX_pta_e',
 'EX_val__L_e',
 'EX_cis_Oleic acid_e',
 'EX_meso_Erythritol_e',
 'EX_inost_e',
 'EX_4hpro_LT_e']

In [165]:
ex_b_reactions = [rxn for rxn in mouse_model_bigg.reactions if rxn.id.startswith('EX_') and rxn.id.endswith('_e')]

#print(f"Found {len(ex_b_reactions)} EX_*_b reactions:")

# Set all lower bounds to 0, with exceptions from diet data
for rxn in ex_b_reactions:
    if rxn.id in RC_bigg_data_convert_df['reaction'].to_list():  # Compare reaction ID, not object
        #print('yes - found in diet')
        # Fix the loc indexing - need to find the row where reaction matches
        #flux_value = temp_media[rxn.id]
        #rxn.lower_bound = -100.0#-1.0*flux_value
        rxn.lower_bound = -1.0*RC_bigg_data_convert_df[RC_bigg_data_convert_df['reaction'] == rxn.id]['fluxValue'].iloc[0]
        #print(f"Setting {rxn.id} lower bound to {rxn.lower_bound}")
    else:
        #print(f"Setting {rxn.id} lower bound from {rxn.lower_bound} to 0")
        rxn.lower_bound = 0

In [166]:
RC_bigg_data_convert_df

,reaction,fluxValue
2,EX_5oxpro_e,0.131
3,EX_ala__L_e,0.836
4,EX_all__D_e,0.029
5,EX_arab__L_e,0.055
7,EX_asp__L_e,0.398
...,...,...
14,EX_na1_e,1.000
15,EX_fe2_e,1.000
16,EX_fe3_e,1.000
17,EX_mobd_e,1.000


In [167]:
mouse_model_bigg.optimize()

,fluxes,reduced_costs
EX_5fthf_e,0.0,0.000000e+00
EX_5homeprazole_e,0.0,0.000000e+00
EX_34dhoxpeg_e,0.0,0.000000e+00
EX_5htrp_e,0.0,0.000000e+00
EX_34dhphe_e,0.0,0.000000e+00
...,...,...
YVITEt,0.0,0.000000e+00
BIOMASS_mm_1_no_glygln,0.0,4.734649e-17
R03184_PLUS_R00647,0.0,0.000000e+00
R01465m,0.0,0.000000e+00


In [168]:
mouse_model_bigg_base.medium

{'EX_arg__L_e': 1.0,
 'EX_ca2_e': 1.0,
 'EX_cl_e': 1.0,
 'EX_co_e': 1.0,
 'EX_co2_e': 100.0,
 'EX_fe2_e': 1.0,
 'EX_fe3_e': 1.0,
 'EX_phe__L_e': 1.0,
 'EX_pi_e': 100.0,
 'EX_h_e': 100.0,
 'EX_h2o_e': 100.0,
 'EX_glc__D_e': 1.0,
 'EX_hco3_e': 100.0,
 'EX_glyc_e': 1.0,
 'EX_hdca_e': 1.0,
 'EX_sel_e': 1.0,
 'EX_so4_e': 100.0,
 'EX_his__L_e': 1.0,
 'EX_trp__L_e': 1.0,
 'EX_i_e': 1.0,
 'EX_val__L_e': 1.0,
 'EX_thr__L_e': 1.0,
 'EX_ile__L_e': 1.0,
 'EX_k_e': 1.0,
 'EX_leu__L_e': 1.0,
 'EX_lnlc_e_copy1': 1.0,
 'EX_lnlc_e_copy2': 1.0,
 'EX_lys__L_e': 1.0,
 'EX_met__L_e': 1.0,
 'EX_na1_e': 1.0,
 'EX_nh4_e': 100.0,
 'EX_o2_e': 100.0}

In [169]:
mouse_model_bigg.medium

{'EX_ala__L_e': 0.836,
 'EX_arab__L_e': 0.055,
 'EX_asp__L_e': 0.398,
 'EX_cys__L_e': 1.989,
 'EX_ca2_e': 1.0,
 'EX_cit_e': 1.046,
 'EX_cl_e': 1.0,
 'EX_co2_e': 1.0,
 'EX_octa_e': 0.13,
 'EX_fe2_e': 1.0,
 'EX_fe3_e': 1.0,
 'EX_phe__L_e': 0.115,
 'EX_pheme_e': 1.0,
 'EX_fru_e': 2.506,
 'EX_pi_e': 1.0,
 'EX_h_e': 1.0,
 'EX_pro__L_e': 2.307,
 'EX_h2o_e': 1.0,
 'EX_gal_e': 0.048,
 'EX_pyr_e': 0.032,
 'EX_glc__D_e': 0.612,
 'EX_glu__L_e': 0.017,
 'EX_gly_e': 0.072,
 'EX_hdca_e': 0.599,
 'EX_ser__D_e': 0.211,
 'EX_so4_e': 1.0,
 'EX_succ_e': 0.056,
 'EX_sucr_e': 32.513,
 'EX_ttdca_e': 0.696,
 'EX_tyr__L_e': 0.093,
 'EX_val__L_e': 0.121,
 'EX_thr__L_e': 0.033,
 'EX_ile__L_e': 0.387,
 'EX_inost_e': 0.502,
 'EX_k_e': 1.0,
 'EX_leu__L_e': 0.644,
 'EX_met__L_e': 4.781,
 'EX_na1_e': 1.0,
 'EX_nac_e': 0.172,
 'EX_nh4_e': 1.0}

In [75]:
only_in_original = mouse_model_bigg_base.medium.keys() - mouse_model_bigg.medium.keys()

In [76]:
only_in_new = mouse_model_bigg.medium.keys() - mouse_model_bigg_base.medium.keys()

In [77]:
### based on these things 

only_in_original

{'EX_arg__L_e',
 'EX_co_e',
 'EX_glyc_e',
 'EX_hco3_e',
 'EX_his__L_e',
 'EX_i_e',
 'EX_lnlc_e_copy1',
 'EX_lnlc_e_copy2',
 'EX_lys__L_e',
 'EX_o2_e',
 'EX_sel_e',
 'EX_trp__L_e'}

In [78]:
test = RC_bigg_data_convert_df['reaction'].sort_values().to_list()

In [81]:
RC_bigg_data_convert_df['reaction'].to_list()

['EX_5oxpro_e',
 'EX_ala__L_e',
 'EX_all__D_e',
 'EX_arab__L_e',
 'EX_asp__L_e',
 'EX_cit_e',
 'EX_cys__L_e',
 'EX_dca_e',
 'EX_dchac_e',
 'EX_ddca_e',
 'EX_fru_e',
 'EX_fum_e',
 'EX_gal_e',
 'EX_glc__D_e',
 'EX_glu__L_e',
 'EX_gly_e',
 'EX_hxa_e',
 'EX_ile__L_e',
 'EX_leu__L_e',
 'EX_lnlcACP_e',
 'EX_mal__L_e',
 'EX_melib_e',
 'EX_met__L_e',
 'EX_ttdca_e',
 'EX_nac_e',
 'EX_octa_e',
 'EX_hdca_e',
 'EX_phe__L_e',
 'EX_C10164_e',
 'EX_pro__L_e',
 'EX_pyr_e',
 'EX_raffin_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_succ_e',
 'EX_sucr_e',
 'EX_thr__L_e',
 'EX_tyr__L_e',
 'EX_pta_e',
 'EX_val__L_e',
 'EX_cis_Oleic acid_e',
 'EX_meso_Erythritol_e',
 'EX_inost_e',
 'EX_4hpro_LT_e',
 'EX_h2o_e',
 'EX_pi_e',
 'EX_nh4_e',
 'EX_mn2_e',
 'EX_zn2_e',
 'EX_so4_e',
 'EX_cu2_e',
 'EX_ca2_e',
 'EX_h_e',
 'EX_cl_e',
 'EX_co2_e',
 'EX_k_e',
 'EX_ni2_e',
 'EX_mg2_e',
 'EX_na1_e',
 'EX_fe2_e',
 'EX_fe3_e',
 'EX_mobd_e',
 'EX_pheme_e']

In [170]:
### ok add his__L, trp__L, lys__L, o2, 
mets_to_add = ['EX_his__L_e', 'EX_trp__L_e', 'EX_lys__L_e', 'EX_o2_e']
flux = [1,1,1,1]

#mets_to_add =['EX_arg__L_e','EX_co_e','EX_glyc_e','EX_hco3_e','EX_his__L_e','EX_i_e','EX_lnlc_e_copy1','EX_lnlc_e_copy2','EX_lys__L_e','EX_o2_e','EX_sel_e','EX_trp__L_e']

#flux = [1,1,1,1,1,1,1,1,1,1,1,1]

mets_to_add_df = pd.DataFrame(mets_to_add, flux)
mets_to_add_df.reset_index(inplace=True)
mets_to_add_df.columns = ['fluxValue','reaction']
mets_to_add_df


,fluxValue,reaction
0,1,EX_his__L_e
1,1,EX_trp__L_e
2,1,EX_lys__L_e
3,1,EX_o2_e


In [233]:
RC_bigg_data_convert_df_remove_added = RC_bigg_data_convert_df[RC_bigg_data_convert_df['fluxValue'] != 1.0]

In [171]:
RC_diet_mouse_additions = pd.concat([RC_bigg_data_convert_df, mets_to_add_df])
RC_diet_mouse_additions

,reaction,fluxValue
2,EX_5oxpro_e,0.131
3,EX_ala__L_e,0.836
4,EX_all__D_e,0.029
5,EX_arab__L_e,0.055
7,EX_asp__L_e,0.398
...,...,...
18,EX_pheme_e,1.000
0,EX_his__L_e,1.000
1,EX_trp__L_e,1.000
2,EX_lys__L_e,1.000


In [172]:

ex_b_reactions = [rxn for rxn in mouse_model_bigg.reactions if rxn.id.startswith('EX_') and rxn.id.endswith('_e')]

#print(f"Found {len(ex_b_reactions)} EX_*_b reactions:")

# Set all lower bounds to 0, with exceptions from diet data
for rxn in ex_b_reactions:
    if rxn.id in RC_diet_mouse_additions['reaction'].to_list():  # Compare reaction ID, not object
        #print('yes - found in diet')
        # Fix the loc indexing - need to find the row where reaction matches
        #flux_value = temp_media[rxn.id]
        #rxn.lower_bound = -100.0#-1.0*flux_value
        rxn.lower_bound = -1.0*RC_diet_mouse_additions[RC_diet_mouse_additions['reaction'] == rxn.id]['fluxValue'].iloc[0]
        #print(f"Setting {rxn.id} lower bound to {rxn.lower_bound}")
    else:
        #print(f"Setting {rxn.id} lower bound from {rxn.lower_bound} to 0")
        rxn.lower_bound = 0


In [173]:
sol = mouse_model_bigg.optimize()

In [174]:
sol

,fluxes,reduced_costs
EX_5fthf_e,0.000000,0.000000e+00
EX_5homeprazole_e,0.000000,0.000000e+00
EX_34dhoxpeg_e,0.000000,0.000000e+00
EX_5htrp_e,0.000000,0.000000e+00
EX_34dhphe_e,0.000000,0.000000e+00
...,...,...
YVITEt,0.000000,0.000000e+00
BIOMASS_mm_1_no_glygln,0.290865,8.853602e-17
R03184_PLUS_R00647,0.000000,0.000000e+00
R01465m,-0.079274,0.000000e+00


In [175]:
mouse_model_bigg.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ala__L_e,EX_ala__L_e,0.5314,0,0.00%
asp__L_e,EX_asp__L_e,0.01104,0,0.00%
cit_e,EX_cit_e,0.2817,0,0.00%
cys__L_e,EX_cys__L_e,0.04218,0,0.00%
fe3_e,EX_fe3_e,1,0,0.00%
fru_e,EX_fru_e,2.506,0,0.00%
glc__D_e,EX_glc__D_e,0.612,0,0.00%
gly_e,EX_gly_e,0.072,0,0.00%
h2o_e,EX_h2o_e,0.2553,0,0.00%
hdca_e,EX_hdca_e,0.07155,0,0.00%


In [39]:
#fva_mouse_model = cobra.flux_analysis.flux_variability_analysis(mouse_model_bigg, loopless=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-05-30
Read LP format model from file /var/folders/5n/y6k0c8y951n38r8ccc6trh1m0000gn/T/tmpe9wajaba.lp
Reading time = 0.01 seconds
: 2776 rows, 7453 columns, 28109 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2026-05-30
Read LP format model from file /var/folders/5n/y6k0c8y951n38r8ccc6trh1m0000gn/T/tmpu56i51xq.lp
Reading time = 0.01 seconds
: 2776 rows, 7453 columns, 28109 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2026-05-30
Read LP format model from file /var/folders/5n/y6k0c8y951n38r8ccc6trh1m0000gn/T/tmpnmq6c34z.lp
Reading time = 0.01 seconds
: 2776 rows, 7453 columns, 28109 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2026-05-30
Read LP format model from file /var/folders/5n/y6k0c8y951n38r8ccc6trh1m0000gn/T/tmpqb3qx4zk.lp
Reading time = 0.01 seconds
: 2776 rows, 7453 

In [176]:
pfba_mouse_model = cobra.flux_analysis.pfba(mouse_model_bigg)

In [177]:
temp_pfba_df = pfba_mouse_model.to_frame().filter(regex='EX_', axis = 0)
temp_pfba_df[temp_pfba_df['fluxes'] != 0.0]

,fluxes,reduced_costs
EX_ala__L_e,-0.531443,2.000000
EX_asp__L_e,-0.011043,2.000000
EX_cys__L_e,-0.042175,2.000000
EX_acac_e,0.086070,-2.000000
EX_cit_e,-0.281742,2.000000
EX_co2_e,0.592870,-2.000000
EX_fe2_e,1.000000,-2.000000
EX_fe3_e,-1.000000,2.161270
EX_for_e,0.005236,-2.000000
EX_phe__L_e,-0.063700,2.000000


In [178]:
#fva_mouse_model_filt = fva_mouse_model.filter(regex='EX_', axis = 0)
#fva_mouse_model_filt[fva_mouse_model_filt['minimum'] != 0.0]

In [179]:
#mouse_model_bigg.optimize()
#mouse_model_bigg.summary(fva=0.95)

In [214]:
#mouse_model_geo_FBA = cobra.flux_analysis.geometric_fba(mouse_model_bigg)
## Here need to set a minimal threshold for objective function 

# Step 1: Get max objective value
solution = mouse_model_bigg.optimize()
max_obj = solution.objective_value

# Step 2: Set a constraint for minimum objective
#objective_rxn = list(linear_reaction_coefficients(model).keys())[0]
mouse_model_bigg.reactions.BIOMASS_mm_1_no_glygln.lower_bound = 0.9 * max_obj

In [215]:
mouse_model_bigg.reactions.BIOMASS_mm_1_no_glygln.lower_bound

0.26177884615384617

In [216]:
flux_samp_df = cobra.sampling.sample(mouse_model_bigg, 10000, processes=10)

Read LP format model from file /var/folders/5n/y6k0c8y951n38r8ccc6trh1m0000gn/T/tmpgvvcjl4d.lp
Reading time = 0.01 seconds
: 2775 rows, 7452 columns, 28106 nonzeros


In [217]:
flux_samp_df

,EX_5fthf_e,EX_5homeprazole_e,EX_34dhoxpeg_e,EX_5htrp_e,EX_34dhphe_e,EX_35cgmp_e,EX_3aib_e,EX_3aib__D_e,EX_3mlda_e,EX_4abut_e,...,XYLTer,XYLTt,XYLUR,XYLt,XYLtly,YVITEt,BIOMASS_mm_1_no_glygln,R03184_PLUS_R00647,R01465m,thr_mt
0,1.149194e-21,0.0,0.0,0.005449,0.0,1.169122e-07,1.479094e-12,2.035754e-11,6.151004e-08,0.000017,...,0.0,-0.000574,0.003681,-7.001827e-22,7.001827e-22,0.0,0.261779,1.474681e-09,-0.101589,-0.101589
1,3.203482e-23,0.0,0.0,0.004650,0.0,1.168023e-07,2.197792e-13,5.674848e-13,5.869019e-07,0.000084,...,0.0,-0.000966,0.004135,-1.951823e-23,1.951823e-23,0.0,0.261779,1.164783e-09,-0.101460,-0.101460
2,2.785502e-20,0.0,0.0,0.000507,0.0,1.082879e-07,3.579305e-13,2.469021e-12,5.543413e-05,0.000069,...,0.0,-0.000963,0.004274,-8.492019e-23,8.492019e-23,0.0,0.261779,4.229158e-04,-0.100593,-0.100593
3,1.997199e-16,0.0,0.0,0.000463,0.0,5.675289e-06,1.736170e-12,1.802014e-07,6.107785e-05,0.000070,...,0.0,-0.000959,0.004957,-7.373591e-22,7.373591e-22,0.0,0.261779,4.359508e-04,-0.100592,-0.100592
4,1.996763e-16,0.0,0.0,0.000397,0.0,1.162139e-02,3.755387e-07,4.718559e-08,6.068233e-05,0.000140,...,0.0,-0.000853,0.004719,2.992625e-17,-2.992625e-17,0.0,0.261791,4.243935e-04,-0.100655,-0.100655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.781391e-13,0.0,0.0,0.071934,0.0,6.132209e-03,5.246835e-05,7.980884e-04,9.233838e-03,0.009818,...,0.0,-0.134410,0.514724,1.787556e-13,-1.787556e-13,0.0,0.264702,2.454036e-02,-0.126710,-0.126710
9996,2.771453e-13,0.0,0.0,0.072361,0.0,7.127561e-03,5.202776e-05,7.308664e-04,9.245178e-03,0.010935,...,0.0,-0.135427,0.514680,1.784357e-13,-1.784357e-13,0.0,0.264701,2.485879e-02,-0.131291,-0.131291
9997,2.685156e-13,0.0,0.0,0.072397,0.0,7.085975e-03,5.204377e-05,7.310290e-04,9.105406e-03,0.010359,...,0.0,-0.135824,0.526742,1.784474e-13,-1.784474e-13,0.0,0.264690,2.480964e-02,-0.129257,-0.129257
9998,2.687795e-13,0.0,0.0,0.067552,0.0,7.001476e-03,5.105716e-05,7.210150e-04,7.121461e-03,0.018099,...,0.0,-0.132868,0.528500,1.777287e-13,-1.777287e-13,0.0,0.264731,3.059729e-02,-0.127285,-0.127285


In [218]:
flux_samp_df = flux_samp_df.T
flux_samp_df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
EX_5fthf_e,1.149194e-21,3.203482e-23,2.785502e-20,1.997199e-16,1.996763e-16,1.996720e-16,1.996773e-16,1.835116e-16,1.835146e-16,1.835246e-16,...,2.428235e-13,2.428178e-13,2.691482e-13,2.684686e-13,2.779688e-13,2.781391e-13,2.771453e-13,2.685156e-13,2.687795e-13,2.687119e-13
EX_5homeprazole_e,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
EX_34dhoxpeg_e,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
EX_5htrp_e,5.449267e-03,4.649969e-03,5.069623e-04,4.631594e-04,3.970770e-04,1.363471e-04,2.024252e-04,2.023925e-04,1.960040e-04,1.961430e-04,...,7.340057e-02,7.340779e-02,7.338633e-02,7.544645e-02,7.484436e-02,7.193386e-02,7.236140e-02,7.239687e-02,6.755232e-02,6.764827e-02
EX_34dhphe_e,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YVITEt,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
BIOMASS_mm_1_no_glygln,2.617788e-01,2.617789e-01,2.617789e-01,2.617789e-01,2.617913e-01,2.617897e-01,2.617901e-01,2.617895e-01,2.617866e-01,2.617793e-01,...,2.643018e-01,2.643020e-01,2.644166e-01,2.646652e-01,2.646909e-01,2.647024e-01,2.647009e-01,2.646903e-01,2.647310e-01,2.647249e-01
R03184_PLUS_R00647,1.474681e-09,1.164783e-09,4.229158e-04,4.359508e-04,4.243935e-04,3.170373e-04,3.105393e-04,3.299032e-04,2.903993e-04,3.322527e-04,...,2.682152e-02,2.678133e-02,2.830101e-02,2.839749e-02,2.628124e-02,2.454036e-02,2.485879e-02,2.480964e-02,3.059729e-02,2.833832e-02
R01465m,-1.015890e-01,-1.014600e-01,-1.005929e-01,-1.005918e-01,-1.006555e-01,-1.006055e-01,-1.006732e-01,-1.006651e-01,-1.018832e-01,-1.016727e-01,...,-1.545229e-01,-1.524685e-01,-1.355802e-01,-1.271631e-01,-1.244604e-01,-1.267102e-01,-1.312913e-01,-1.292567e-01,-1.272846e-01,-1.447880e-01


In [219]:
flux_samp_df_filt = flux_samp_df.filter(regex = 'EX_', axis = 0)
flux_samp_df_filt_mean = flux_samp_df_filt.mean(axis=1).to_frame()
flux_samp_df_filt_mean.columns = ['mean_flux']
flux_samp_df_filt_mean_up = flux_samp_df_filt_mean[flux_samp_df_filt_mean['mean_flux'] < -1e-10]
flux_samp_df_filt_mean_sec = flux_samp_df_filt_mean[flux_samp_df_filt_mean['mean_flux'] > 1e-3]

In [220]:
flux_samp_df_filt_mean_sec

,mean_flux
EX_5htrp_e,0.019013
EX_35cgmp_e,0.012341
EX_3aib__D_e,0.001328
EX_3mlda_e,0.009379
EX_4abut_e,0.077872
...,...
EX_mag_hs_e,0.059552
EX_man_e,0.013341
EX_mthgxl_e,1.981126
EX_no_e,0.025286


In [221]:
flux_samp_df_filt_mean_up

,mean_flux
EX_ala__L_e,-0.462812
EX_arab__L_e,-0.040488
EX_asp__L_e,-0.349408
EX_cys__L_e,-0.066429
EX_cit_e,-0.763369
EX_octa_e,-0.040575
EX_fe3_e,-0.823090
EX_phe__L_e,-0.068664
EX_fru_e,-2.147254
EX_pi_e,-0.727273


In [222]:

### ok now filter the uptake and secretion dataframe by the ~60 compounds looked at in metabolomics
all_mets_bigg = []
for i in rc_diet_data.columns.tolist()[1:]:
    all_mets_bigg.append('EX_' + i + '_e')

In [225]:
flux_samp_df_filt_mean_up = flux_samp_df_filt_mean_up[flux_samp_df_filt_mean_up.index.isin(all_mets_bigg)]

In [226]:
flux_samp_df_filt_mean_sec = flux_samp_df_filt_mean_sec[flux_samp_df_filt_mean_sec.index.isin(all_mets_bigg)]

In [231]:
### concat the filtered uptake and secretion 

flux_samp_df_filt_concat = pd.concat([flux_samp_df_filt_mean_up, flux_samp_df_filt_mean_sec])
flux_samp_df_filt_concat = flux_samp_df_filt_concat.reset_index()
flux_samp_df_filt_concat.columns = ['reaction', 'fluxValue']

In [234]:
### Wait get rid of the added fluxes here... 
RC_bigg_data_convert_df_remove_added

,reaction,fluxValue
2,EX_5oxpro_e,0.131
3,EX_ala__L_e,0.836
4,EX_all__D_e,0.029
5,EX_arab__L_e,0.055
7,EX_asp__L_e,0.398
10,EX_cit_e,1.046
11,EX_cys__L_e,1.989
12,EX_dca_e,0.046
13,EX_dchac_e,0.028
14,EX_ddca_e,0.038


In [235]:
mouse_GEM_diet_changes = pd.concat([flux_samp_df_filt_concat, RC_bigg_data_convert_df_remove_added])

# Group by reaction and sum
mouse_GEM_diet_changes = mouse_GEM_diet_changes.groupby('reaction', as_index=False).sum()
mouse_GEM_diet_changes
### so seems like lys and his are added in negative amounts because they aren't in dietary stuff in general, but just remove them for now, these are the only two negatives technically


,reaction,fluxValue
0,EX_4hpro_LT_e,0.004000
1,EX_5oxpro_e,0.131000
2,EX_C10164_e,0.064000
3,EX_akg_e,0.342661
4,EX_ala__L_e,0.373188
5,EX_all__D_e,0.029000
6,EX_arab__L_e,0.014512
7,EX_asn__L_e,0.032044
8,EX_asp__L_e,0.048592
9,EX_cis_Oleic acid_e,3.842000


In [238]:
### remove those negative values

mouse_GEM_diet_changes = mouse_GEM_diet_changes[mouse_GEM_diet_changes['fluxValue'] > 0]
mouse_GEM_diet_changes.reset_index(inplace=True)

In [239]:
### Convert dietary reaction names back to kbase for easy importing to MDSINE-FBA
kbase_names_for_final_diet = []

for i in range(0, len(mouse_GEM_diet_changes)):
    print(mouse_GEM_diet_changes['reaction'].loc[i])

EX_4hpro_LT_e
EX_5oxpro_e
EX_C10164_e
EX_akg_e
EX_ala__L_e
EX_all__D_e
EX_arab__L_e
EX_asn__L_e
EX_asp__L_e
EX_cis_Oleic acid_e
EX_cit_e
EX_cys__L_e
EX_dca_e
EX_dchac_e
EX_ddca_e
EX_drib_e
EX_fru_e
EX_fuc__L_e
EX_fum_e
EX_gal_e
EX_glc__D_e
EX_glu__L_e
EX_gly_e
EX_hdca_e
EX_hxa_e
EX_ile__L_e
EX_inost_e
EX_leu__L_e
EX_lnlcACP_e
EX_mal__L_e
EX_melib_e
EX_meso_Erythritol_e
EX_met__L_e
EX_nac_e
EX_octa_e
EX_phe__L_e
EX_pro__L_e
EX_pta_e
EX_pyr_e
EX_raffin_e
EX_rmn_e
EX_ser__D_e
EX_succ_e
EX_sucr_e
EX_thr__L_e
EX_ttdca_e
EX_tyr__L_e
EX_val__L_e
